In [ ]:
# from enhanced_eps_extraction import extract_eps_with_requests_html_chunked 

# grouped_df = pd.read_excel('grouped_df.xlsx',dtype=str)
# grouped_df['xbrl'] = grouped_df['xbrl'].apply(lambda x: eval(x))

# result = extract_eps_with_requests_html_chunked(grouped_df , max_workers=5)
# result.to_excel('result.xlsx',index=False)


In [3]:
import pandas as pd
import requests , json 
import concurrent.futures
import threading
from tqdm import tqdm




In [9]:
result = pd.read_excel('result.xlsx',dtype=str)
result.fillna('',inplace=True)
result['eps'] = result['eps'].apply(lambda x: eval(x) if '[' in x else [])

<h1>Computing the avg eps , removing None type eps(failed to extract)

In [10]:
result['avg_eps'] = result.apply(
    lambda x: sum(
        [float(item) for item in x['eps'] if item is not None and isinstance(item, (int, float,str))]
    ) / len(
        [float(item) for item in x['eps'] if item is not None and isinstance(item, (int, float ,str))]
    ) if len([float(item) for item in x['eps'] if item is not None and isinstance(item, (int, float ,str))]) > 0 else "",
    axis=1
)

In [11]:
result['total_eps'] = result.apply(
    lambda x: len([float(item) for item in x['eps'] if item is not None and isinstance(item, (int, float, str))]),
    axis=1
)

In [12]:
result['3Yr_avg_eps'] = result.apply(
    lambda x: sum(
        [float(item) for item in x['eps'] if item is not None and isinstance(item, (int, float,str))][:3]
    ) / 3 if len([float(item) for item in x['eps'] if item is not None and isinstance(item, (int, float,str))]) >= 3 else "", 
    axis=1
)

In [13]:
seven_and_above = result[result['total_eps']>=7].copy(deep=True)
three = result[(result['3Yr_avg_eps']!="")].copy(deep=True)
three = three[(three['3Yr_avg_eps']>0)].copy(deep=True)


In [14]:
three['fifftee_times']  = ""
three['fifftee_times'] = three.apply(lambda x: 15*x['3Yr_avg_eps'],axis=1)


three.reset_index(drop=True,inplace=True)
three['cmp'] = ""

In [15]:
# def get_price(symbol):
#     url = f"https://www.nseindia.com/api/quote-equity?symbol={symbol}"

#     payload = {}
#     headers = {
#     'accept': '*/*',
#     'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
#     'cookie': cookie,
#     'priority': 'u=1, i',
#     'referer': 'https://www.nseindia.com/get-quotes/equity?symbol=RELIANCE',
#     'sec-ch-ua': '"Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"',
#     'sec-ch-ua-mobile': '?0',
#     'sec-ch-ua-platform': '"macOS"',
#     'sec-fetch-dest': 'empty',
#     'sec-fetch-mode': 'cors',
#     'sec-fetch-site': 'same-origin',
#     'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36'
#     }

#     response = requests.request("GET", url, headers=headers, data=payload)

#     return json.loads(response.text)['priceInfo']['lastPrice']


# for i in tqdm(range(three.shape[0])):
#     three.loc[i,'cmp'] = get_price(three.loc[i,'symbol'])


In [16]:
import pandas as pd
import requests
import json
from tqdm import tqdm
import time
from requests_html import HTMLSession
import os
from fake_useragent import UserAgent

from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium_stealth import stealth
# import undetected_chromedriver as uc
from bs4 import BeautifulSoup
from enhanced_eps_extraction import extract_eps_with_requests_html_chunked 



chrome_options = Options()
# chrome_options.add_argument("--headless")  # Enable headless mode
chrome_options.add_argument("--no-sandbox")  # Required for some environments
chrome_options.add_argument("--disable-dev-shm-usage")  # Helps in Docker environments

ua = UserAgent()
fake_user_agent = ua.random
chrome_options.add_argument(f"--user-agent={fake_user_agent}")

from bs4 import BeautifulSoup

driver = webdriver.Chrome(options=chrome_options)
stealth(driver,
        # user_agent='your-custom-user-agent',  # Set custom user agent
        languages=["en-US", "en"],  # Set languages
        vendor="Mozilla",  # Set vendor
        platform="Win32",  # Set platform
        webgl_vendor="Mozilla",  # Set WebGL vendor
        renderer="Mozilla",  # Set WebGL renderer
        fix_hairline=True)  # Fix hairline issue in headless mode
driver.get('https://www.nseindia.com/market-data/52-week-low-equity-market')
cookies = driver.get_cookies()
time.sleep(2)


cookie = ""
for item in driver.get_cookies():
    cookie += item['name'] + "=" + item['value'] + "; "

In [44]:
# driver.quit()

In [17]:
# symbols_with_indices[0]

In [18]:
cookie

'RT="z=1&dm=nseindia.com&si=3cc73d83-c91f-47b2-ac50-c27c960e44c9&ss=mc9gooip&sl=1&se=8c&tt=qt&bcn=%2F%2F684d0d46.akstat.io%2F&ld=22l"; _abck=29C35CF8D90A7F33522DB745B660A5F0~-1~YAAQLagRYC/Li4GXAQAAKq8png5Kuyj4UDIaLdj6BIiD6PsHuHdhpXsPW/6KO/uX60g7dYNUID1DEjk7kybVx55UsOgbVauvd1mb1qb7pm7xCJsez2Ua+nwcHi+WCXd2l6Oh5f100AVGI1LiGcqBWENEU1WjrFtBl+falRcpIaNk+F1XCTVQZZUKhf54g5k237vP9uC54EDcSc80I449B6lpnNHh5MVeU+qDpGcK8iTtsPetSfdeDUO+bmcrdhLmaQcPHq3Yo3jmJbd7ca8AS6lNfFYHyDFjO25D6VpQcIxRF+sjibbZLvnKxybr006BYZeoHNApeS9PG0hHrz7PyLUYZ3lBThG1fQ/El2+3z/210VOCB876digzJeNpjnWRaTlnGTWcLFHbXgT5kCZ6iHxPqC5xDeIg3Lb+RqHd0jvMKtSt1qGuRveZHpmmNqrklbaxtAFWYyZyEBe5j3JNHG6CF0/DV5J7RJ3UsUcDHv39/FXYLxttEoUEwdY1uINn3njfA5F3dXD+r+MjJotEF36B/z8LPlcUqf/63Z6TLvm9l1G/nOHTMsB6jVd74sADjeh5twcPYk0jRigPMNhEw+n0IGsE~-1~||0||~-1; ak_bmsc=3F6211CF274875AFAFFC61EE09299EE8~000000000000000000000000000000~YAAQLagRYArLi4GXAQAARKwpnhx1ubGjH99YyqAdf2bYEnxIto3ukpOqd4GxBYXOOcRu82U6GEEA+w4mxkbDb+ocHvrSjhqXg7KcEz7vtWXZlCwX1+KPKsxugAV/uOPQKPh8L

In [20]:
url = f"https://www.nseindia.com/api/quote-equity?symbol=20MICRONS"

payload = {}
headers = {
'accept': '*/*',
'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
'cookie': cookie,
'priority': 'u=1, i',
'referer': f'https://www.nseindia.com/get-quotes/equity?symbol=20MICRONS',
'sec-ch-ua': '"Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"',
'sec-ch-ua-mobile': '?0',
'sec-ch-ua-platform': '"macOS"',
'sec-fetch-dest': 'empty',
'sec-fetch-mode': 'cors',
'sec-fetch-site': 'same-origin',
'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36'
}

response = requests.request("GET", url, headers=headers, data=payload, timeout=30)
json.loads(response.text)['priceInfo']['lastPrice']

222.6

In [21]:
# Multithreaded version of price fetching - Copy this into your notebook cell

def get_price_with_error_handling(symbol):
    """Get price for a symbol with error handling"""
    try:
        url = f"https://www.nseindia.com/api/quote-equity?symbol={symbol}"

        payload = {}
        headers = {
        'accept': '*/*',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
        'cookie': cookie,
        'priority': 'u=1, i',
        'referer': f'https://www.nseindia.com/get-quotes/equity?symbol={symbol}',
        'sec-ch-ua': '"Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36'
        }

        response = requests.request("GET", url, headers=headers, data=payload, timeout=30)
        return json.loads(response.text)['priceInfo']['lastPrice']
    except Exception as e:
        print(f"Error fetching price for {symbol}: {e}")
        return None

# Get list of symbols and their indices
symbols_with_indices = [(i, symbol) for i, symbol in enumerate(three['symbol'])]

# Use ThreadPoolExecutor for parallel price fetching
max_workers = 10  # Adjust based on your system and API limits
prices_dict = {}

with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    # Submit all tasks
    future_to_index = {executor.submit(get_price_with_error_handling, symbol): (i, symbol) 
                      for i, symbol in symbols_with_indices}
    
    # Collect results with progress bar
    for future in tqdm(concurrent.futures.as_completed(future_to_index), 
                      total=len(symbols_with_indices),
                      desc="Fetching prices (multithreaded)"):
        index, symbol = future_to_index[future]
        try:
            price = future.result()
            prices_dict[index] = price
        except Exception as e:
            print(f"Error processing {symbol}: {e}")
            prices_dict[index] = None

# Update the DataFrame with fetched prices
for index, price in prices_dict.items():
    three.loc[index, 'cmp'] = price

print(f"Completed fetching prices for {len(prices_dict)} symbols") 

Fetching prices (multithreaded): 100%|██████████| 2482/2482 [01:45<00:00, 23.63it/s]


Completed fetching prices for 2482 symbols


In [22]:
driver.quit()

,symbol,consolidated,xbrl,eps,avg_eps,total_eps,3Yr_avg_eps,fifftee_times,cmp
0,20MICRONS,Consolidated,['https://nsearchives.nseindia.com/corporate/x...,"[15.89, 11.85, 9.80, 6.51, 6.83, 7.06, 5.34]",9.04,7,12.513333,187.70,222.6
1,20MICRONS,Non-Consolidated,['https://nsearchives.nseindia.com/corporate/x...,"[14.21, 10.25, 8.75, 6.36, 7.08, 6.15, 4.50, 4...",7.725,8,11.07,166.05,222.6
2,21STCENMGM,Consolidated,['https://nsearchives.nseindia.com/corporate/x...,"[30.64, -17.20, 13.96, 4.75, -30.55, None, 5.7...",1.857143,7,9.133333,137.00,65.04
3,21STCENMGM,Non-Consolidated,['https://nsearchives.nseindia.com/corporate/x...,"[30.70, -14.84, 14.49, -3.48, -11.29, 5.25, 5....",3.95625,8,10.116667,151.75,65.04
4,360ONE,Consolidated,['https://nsearchives.nseindia.com/corporate/x...,"[22.48, 18.51, 65.41, 23.37]",32.4425,4,35.466667,532.00,1156
...,...,...,...,...,...,...,...,...,...
2477,ZUARIIND,Non-Consolidated,['https://nsearchives.nseindia.com/corporate/x...,"[7.61, 6.17, 5.10, 16.96, 4.39, 13.68, 6.10]",8.572857,7,6.293333,94.40,283
2478,ZYDUSLIFE,Consolidated,['https://nsearchives.nseindia.com/corporate/x...,"[38.14, 19.30, 43.84, 20.84, 11.49, 18.06, 17.35]",24.145714,7,33.76,506.40,953
2479,ZYDUSLIFE,Non-Consolidated,['https://nsearchives.nseindia.com/corporate/x...,"[34.01, 15.06, 8.38, 14.42, 13.80, 15.65, 10.66]",15.997143,7,19.15,287.25,953
2480,ZYDUSWELL,Consolidated,['https://nsearchives.nseindia.com/corporate/x...,"[41.94, 48.78, 48.54, 19.55, 24.58, 34.27]",36.276667,6,46.42,696.30,2036


In [24]:
three['cmp'] = three['cmp'].apply(lambda x: float(x))

In [25]:
three[three['fifftee_times']>three['cmp']].sort_values(by=['cmp'],inplace=True)

/var/folders/mn/8x0b1x_d67lf_rj9wy4xy5dw0000gn/T/ipykernel_13534/1838620807.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  three[three['fifftee_times']>three['cmp']].sort_values(by=['cmp'],inplace=True)


In [26]:
three[three['fifftee_times']>three['cmp']].sort_values(by=['cmp'])

,symbol,consolidated,xbrl,eps,avg_eps,total_eps,3Yr_avg_eps,fifftee_times,cmp
1342,MACPOWER,Non-Consolidated,['https://nsearchives.nseindia.com/corporate/x...,"[24.24, 12.88, 12.83, 6.03]",13.995,4,16.65,249.75,0.0
2399,VIPULLTD,Consolidated,['https://nsearchives.nseindia.com/corporate/x...,"[19.46, -9.78, -3.49, -4.63, 0.10, 0.00, 0.00]",0.237143,7,2.063333,30.95,0.0
1651,PANACHE,Non-Consolidated,['https://nsearchives.nseindia.com/corporate/x...,"[0.34, 1.54, 1.75, 2.95, 1.70]",1.656,5,1.21,18.15,0.0
2395,VINEETLAB,Non-Consolidated,['https://nsearchives.nseindia.com/corporate/x...,"[1.11, 1.36, 7.16, 2.18]",2.9525,4,3.21,48.15,0.0
1650,PANACHE,Consolidated,['https://nsearchives.nseindia.com/corporate/x...,"[0.46, 0.69, 0.67, 0.30, -2.45]",-0.066,5,0.606667,9.10,0.0
...,...,...,...,...,...,...,...,...,...
482,COFORGE,Non-Consolidated,['https://nsearchives.nseindia.com/corporate/x...,"[161.49, 120.12, 106.19, 39.32, 67.93, 48.55, ...",77.1575,8,129.266667,1939.00,1830.0
357,BIRLANU,Consolidated,['https://nsearchives.nseindia.com/corporate/x...,"[46.15, 129.09, 280.54, 346.89, 141.74]",188.882,5,151.926667,2278.90,2212.0
358,BIRLANU,Non-Consolidated,['https://nsearchives.nseindia.com/corporate/x...,"[136.59, 172.95, 247.73, 250.73, 102.63]",182.126,5,185.756667,2786.35,2212.0
2378,VHL,Consolidated,['https://nsearchives.nseindia.com/corporate/x...,"[579.28, 715.75, 1344.46, 405.80, 500.43, 591.53]",689.541667,6,879.83,13197.45,3967.3


In [28]:
# This will keep only rows where all eps values are non-negative (ignoring None)
non_negative_eps = three[~three['eps'].apply(lambda x: any(float(e) < 0 for e in x if e is not None))]

In [32]:
non_negative_eps.sort_values(by=['cmp']).to_excel('INDIAN_STOCKS_WITH_EPS.xlsx',index=False)